In [1]:
import torchvision
from torchvision import models
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import os, subprocess
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
for name in dir(torchvision.models):
    print(name)

AlexNet
AlexNet_Weights
ConvNeXt
ConvNeXt_Base_Weights
ConvNeXt_Large_Weights
ConvNeXt_Small_Weights
ConvNeXt_Tiny_Weights
DenseNet
DenseNet121_Weights
DenseNet161_Weights
DenseNet169_Weights
DenseNet201_Weights
EfficientNet
EfficientNet_B0_Weights
EfficientNet_B1_Weights
EfficientNet_B2_Weights
EfficientNet_B3_Weights
EfficientNet_B4_Weights
EfficientNet_B5_Weights
EfficientNet_B6_Weights
EfficientNet_B7_Weights
EfficientNet_V2_L_Weights
EfficientNet_V2_M_Weights
EfficientNet_V2_S_Weights
GoogLeNet
GoogLeNetOutputs
GoogLeNet_Weights
Inception3
InceptionOutputs
Inception_V3_Weights
MNASNet
MNASNet0_5_Weights
MNASNet0_75_Weights
MNASNet1_0_Weights
MNASNet1_3_Weights
MaxVit
MaxVit_T_Weights
MobileNetV2
MobileNetV3
MobileNet_V2_Weights
MobileNet_V3_Large_Weights
MobileNet_V3_Small_Weights
RegNet
RegNet_X_16GF_Weights
RegNet_X_1_6GF_Weights
RegNet_X_32GF_Weights
RegNet_X_3_2GF_Weights
RegNet_X_400MF_Weights
RegNet_X_800MF_Weights
RegNet_X_8GF_Weights
RegNet_Y_128GF_Weights
RegNet_Y_16GF_We

In [3]:
import torch
import torch.nn as nn

from tqdm import tqdm

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = models.efficientnet_b0(weights=weights)
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [4]:
model.classifier[1].out_features=20
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [5]:
#parameter 수 확인
total_params = sum(p.numel() for p in model.parameters())
total_params

5288548

In [6]:
data_dir = "./data"
dir_lst = os.listdir(data_dir)
for dir in dir_lst:
    dir_path = os.listdir(f"./data/{dir}")

    for name in dir_path:
        if name == ".ipynb_checkpoints":
            print(name)
            os.rmdir(f"./data/{dir}/{name}")

In [7]:
IMG_TRANSFORM = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((512,512), antialias = True),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
    ])

In [8]:
train_imgfolder = ImageFolder(root="./data/train",
                              transform=IMG_TRANSFORM,
                              target_transform=None
                             )

test_imgfolder = ImageFolder(root="./data/test",
                              transform=IMG_TRANSFORM
                             )

In [9]:
Image.MAX_IMAGE_PIXELS = None

train_dataloader = DataLoader(dataset=train_imgfolder,
							  batch_size = 16,
                              num_workers=os.cpu_count(),
                              shuffle=True,
                              drop_last=True 
                             )

test_dataloader = DataLoader(dataset=test_imgfolder,
							  batch_size = 16,
                              num_workers=os.cpu_count(),
                              shuffle=True,
                              drop_last=True 
                            )

In [10]:
# model setting

EPOCH = 50
LEARNING_RATE = 1e-3 
DEVICE = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay = 0.001)
criterion = nn.CrossEntropyLoss()
print("Using device: ",DEVICE)

Using device:  cuda:2


In [11]:
def loss_fn(class_output, label, criterion):
    CE_loss = criterion(class_output, label)

    return CE_loss 

In [12]:
def train(train_loader, model, optimizer, criterion, epoch):
    model.train()
    correct = 0
    total = 0
    
    epoch_loss = []
    for index, (img_data,labels) in enumerate(train_loader):
        img_data, labels = img_data.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        
        outputs = model(img_data)
        loss = loss_fn(outputs, labels, criterion)
        
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        
        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.item())
        if index%100==0:
            print(f'train loss : {np.mean(epoch_loss):>7f}, [epoch:{epoch}, iter:{index}]')
            
    accuracy = correct / total
    print(f"TRAIN ACCURACY : {(100*accuracy):>0.1f}% [{correct}/{total}]")
    return np.mean(epoch_loss)

In [13]:
def test(test_loader, model, criterion, epoch):
    model.eval()

    epoch_loss = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        for index, (img_data,labels) in enumerate(test_loader):
            img_data, labels = img_data.to(DEVICE), labels.to(DEVICE)
            outputs = model(img_data)
    
            loss = loss_fn(outputs, labels, criterion)

            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            epoch_loss.append(loss.item())
    
    accuracy = correct / total
    
    print(f"TEST ACCURACY : {(100*accuracy):>0.1f}% [{correct}/{total}]")
    return np.mean(epoch_loss)

In [14]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
train_loss_lst, test_loss_lst = [], []

model_version = 1


test_best_loss = 100
test_current_loss = 100
early_stop_threshold = 5
early_stop_trigger = 0

model = model.to(DEVICE)

for i in tqdm(range(EPOCH), desc = 'Train'):
    print(f"EPOCH {i+1} \n-------------------------")
    train_loss = train(train_dataloader, model, optimizer, criterion, i+1)
    test_loss = test(test_dataloader, model, criterion, i+1)

    train_loss_lst.append(train_loss)
    test_loss_lst.append(test_loss)

    if test_loss < test_best_loss :
        print("...MODEL SAVE...")
        test_best_loss = test_loss 
        
        if model_version == 0:
            torch.save(model.state_dict(),'contrastive_model_for_style_weight_v2_without_target_style.pth')
        else:
            torch.save(model.state_dict(),'contrastive_model_for_style_weight_efficientnet_b0.pth')
        
    if test_current_loss < test_loss:
        early_stop_trigger += 1 
    else:
        early_stop_trigger = 0 
    test_current_loss  = test_loss 

    print(f'\nEPOCH:{i+1} | train loss : {train_loss}, test loss : {test_loss}\n')
    
    if early_stop_trigger >= early_stop_threshold:
        break
        
print("DONE!")

Train:   0%|                                       | 0/50 [00:00<?, ?it/s]

EPOCH 1 
-------------------------
train loss : 7.255861, [epoch:1, iter:0]
train loss : 2.672246, [epoch:1, iter:100]
train loss : 2.354867, [epoch:1, iter:200]


/home/qkrwnstj/anaconda3/envs/qkrwnstj/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train loss : 2.195069, [epoch:1, iter:300]
train loss : 2.101297, [epoch:1, iter:400]
train loss : 2.028422, [epoch:1, iter:500]
train loss : 1.989189, [epoch:1, iter:600]
train loss : 1.953131, [epoch:1, iter:700]
train loss : 1.921571, [epoch:1, iter:800]
train loss : 1.899241, [epoch:1, iter:900]
train loss : 1.881528, [epoch:1, iter:1000]
train loss : 1.865716, [epoch:1, iter:1100]
train loss : 1.855430, [epoch:1, iter:1200]
train loss : 1.845064, [epoch:1, iter:1300]


/home/qkrwnstj/anaconda3/envs/qkrwnstj/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


...

train loss : 1.435425, [epoch:13, iter:2200]
train loss : 1.435442, [epoch:13, iter:2300]
train loss : 1.435084, [epoch:13, iter:2400]
train loss : 1.436111, [epoch:13, iter:2500]
train loss : 1.436898, [epoch:13, iter:2600]
train loss : 1.439118, [epoch:13, iter:2700]
train loss : 1.439425, [epoch:13, iter:2800]
train loss : 1.438767, [epoch:13, iter:2900]
train loss : 1.438776, [epoch:13, iter:3000]
train loss : 1.439696, [epoch:13, iter:3100]
train loss : 1.441117, [epoch:13, iter:3200]


/home/qkrwnstj/anaconda3/envs/qkrwnstj/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train loss : 1.441759, [epoch:13, iter:3300]
train loss : 1.443247, [epoch:13, iter:3400]
train loss : 1.443116, [epoch:13, iter:3500]
train loss : 1.444049, [epoch:13, iter:3600]
train loss : 1.443139, [epoch:13, iter:3700]
train loss : 1.443923, [epoch:13, iter:3800]
TRAIN ACCURACY : 53.1% [32779/61680]


Train:  26%|█████▉                 | 13/50 [3:52:43<11:06:36, 1081.00s/it]

TEST ACCURACY : 43.1% [5029/11680]

EPOCH:13 | train loss : 1.4440217598557628, test loss : 1.7511310742325978

EPOCH 14 
-------------------------
train loss : 0.922688, [epoch:14, iter:0]
train loss : 1.421387, [epoch:14, iter:100]
train loss : 1.429680, [epoch:14, iter:200]
train loss : 1.428502, [epoch:14, iter:300]
train loss : 1.428761, [epoch:14, iter:400]
train loss : 1.436018, [epoch:14, iter:500]
train loss : 1.427060, [epoch:14, iter:600]


In [ ]:
figure = plt.figure(figsize = (10,5))
rows, cols = 1, 2 

title = ['train loss', 'test loss']
values = [train_loss_lst, test_loss_lst]
for i in range(1, rows* cols +1):
    figure.add_subplot(rows,cols,i)
    plt.title(title[i-1])
    plt.plot(values[i-1])

plt.show()